# Making a diagnostic
This notebook aims to help you make a new diagnostic function.

In [1]:
from pathlib import Path
from datatree import DataTree
import xarray as xr
import pandas as pd
import valenspy as vp

## Demo data
The demo data set is already in CF convention so no input conversion and preprocessing is needed.

In [27]:
demo_data = vp.demo_data_CF
demo_ds = xr.open_dataset(demo_data)
vp.cf_checks.is_cf_compliant(demo_ds)

True

If more intrecate/complex data is needed to test the new diagnostic we recommend using CMIP6 data from Google cloud.

In [3]:
import pandas as pd
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df

#Querry the data you need
temperature_data = df.query("table_id== 'day' & source_id == 'EC-Earth3-Veg' & variable_id == 'tas' & experiment_id == 'historical' & member_id == 'r1i1p1f1' & activity_id == 'CMIP'")

# Get the zarr store
temperature_data = temperature_data.zstore.values[0]

# Open the dataset
ds_ec_temp = xr.open_zarr(temperature_data)
vp.cf_checks.is_cf_compliant(ds_ec_temp)

True

## Let's make a new diagnostic

To make a new diagnostic, you need to define a 2 functions:
1. `diagnostic` function that takes a dataset and returns a diagnostic value.
2. `diagnostic_plot` function that takes output from the `diagnostic` function and returns a plot.

There are several types of diagnostics, depending on the type different input/outputs are expected. 

### Model2Ref

This compares 1 single model to a reference. Therefore, a Model2Ref diagnostic expects the following inputs:
- `data`: **xarray dataset** of the model data
- `ref`: **xarray dataset** of the reference data

the diagnostic function returns the results (this can be any type of data) and the plot function returns a plot.

#### Let's take a look at the example below

In [10]:

#Diagnostic functions to calculate the area average warming (tas) compared to a reference period and get the time of crossing a certain warming level
def warming_levels(ds, ref, levels=[1.5, 2.0], rol_years=21, freq=None):
    """
    Calculate the crossing times for different warming levels - the time when the area average warming crosses a certain level compared to the reference period

    Parameters
    ----------
    ds : xarray.Dataset
        Dataset containing the model data
    ref : xarray.Dataset
        Dataset containing the reference period
    levels : list
        List of warming levels to get crossing times for
    rol_years : int
        Number of years to use for the rolling mean
    freq : str, optional
        Frequency of the data (following pandas conventions), default is None and will be inferred from the data
    
    Returns
    -------
    crossingtimes : dict
        Dictionary containing the crossing times for each warming level
    temp_warming : xarray.DataArray
        DataArray containing the area average warming compared to the reference period
    """
    warming_ds = temp_warming(ds.resample(time="ME").mean(), ref, rol_amount=rol_years*12)
    crossingtimes = {}
    for level in levels:
        try:
            crossingtimes[level] = warming_ds.where(warming_ds>level, drop=True).idxmin('time').astype('datetime64[ns]').values
        except ValueError:
            crossingtimes[level] = False
    return crossingtimes, warming_ds

def temp_warming(ds, ref, rol_amount):
    """
    Calculate the area average warming compared to the reference period

    Parameters
    ----------
    ds : xarray.Dataset
        Dataset containing the model data
    ref : xarray.Dataset
        Dataset containing the reference period
    rol_years : int
        Number of years to use for the rolling mean
    
    Returns
    -------
    ds_temp_warming : xarray.DataArray
        DataArray containing the area average warming compared to the reference period
    """
    ref_temp = ref.tas.mean()
    rol = ds.tas.rolling(time=rol_amount).mean().mean(dim=['lat','lon'])
    ds_temp_warming = rol - ref_temp

    return ds_temp_warming

Test the diagnostic function with the demo data.

In [11]:
ds_ec_temp

ref_temp = ds_ec_temp.sel(time=slice('1850', '1900'))

In [12]:
ct, tw_ds = warming_levels(ds_ec_temp, ref_temp, levels=[0.4, 1], rol_years=1)

/dodrio/scratch/projects/2022_200/project_output/RMIB-UGent/vsc46032_kobe/valenspy_dev/lib/python3.9/site-packages/xarray/core/groupby.py:668: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  index_grouper = pd.Grouper(


In [ ]:
ct

{0.4: False, 1: False}

In [ ]:
tw_ds

<xarray.DataArray 'tas' (time: 10)> Size: 80B
array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])
Coordinates:
    height   float64 8B 2.0
  * time     (time) datetime64[ns] 80B 1953-12-31 1954-12-31 ... 1962-12-31

### Write your own Model2Ref diagnostic and plotting function

In [8]:
#Note that ds and rf are expected to be xarray datasets, add aditional arguments to your function if needed
def your_diagnostic_function(ds: xr.Dataset, ref: xr.Dataset) -> DataTree: #Replace DataTree with your expected output type!
    pass #Replace this with your code

def your_diagnostic_plotting_function():
    pass #

Test the functions perform as expected with the demo data

In [9]:
your_diagnostic_function(demo_ds, demo_ds)

## Finally make the diagnostic

In [ ]:
#your_diag = vp.Diagnostic(your_diagnostic_function, your_diagnostic_plotting_function, 'Your diagnostic name', 'Your diagnostic description')
example_diag = vp.Diagnostic(warming_levels, None, 'Warming levels', 'Calculate the crossing times for different warming levels')

Apply it

In [ ]:
with ProgressBar():
    #result = your_diag.apply(demo_ds, demo_ds)
    result = example_diag.apply(demo_ds, demo_ds)

NameError: name 'your_diag' is not defined

Plot it

In [ ]:
#your_diag.plot(result)
example_diag.plot(result)

## Congratulations! You have made a new diagnostic function!
You can now add it to the diagnostics_functions.py file so everyone can use it.

TODO: short guideline on how to add a new diagnostic function to the diagnostics_functions.py file.